
<font size="5">**<center> ĐỒ ÁN CUỐI KỲ </center>** </font>

<font size="4">**<center> KHOA HỌC DỮ LIỆU</center>** </font>

# I/ Thông tin nhóm:

| STT |       Tên       |   MSSV  |
|-----|:---------------:|--------:|
|  1  |   Thái Bá Sơn   | 1712732 |
|  2  | Huỳnh Công Sinh | 1712724 |

# II/ Khai thác dữ liệu:

In [ ]:
# from selenium import webdriver
import time
# from selenium.common.exceptions import NoSuchElementException
# from selenium.webdriver.common.action_chains import ActionChains
# from selenium.webdriver.common.keys import Keys
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

Dữ liệu khai thác: [tại đây](https://www.nasdaq.com/) 

Cấu trúc câu lệnh khai thác: 'https://www.nasdaq.com/market-activity/stocks/' + 'stock_code' + '/historical'


# I/ Dữ liệu

### Thông tin về dữ liệu:


Trong sổ tay này, chúng ta sẽ xem xét dữ liệu từ thị trường chứng khoán, đặc biệt là một số cổ phiếu công nghệ. Sử dụng pandas để lấy thông tin cổ phiếu, hình dung các khía cạnh khác nhau của nó và cuối cùng ta sẽ xem xét một số cách phân tích rủi ro của cổ phiếu, dựa trên lịch sử hoạt động trước đó của nó. Chúng tôi cũng sẽ dự đoán giá cổ phiếu trong tương lai thông qua phương pháp Long Short Term Memory (LSTM)! 

**Nguồn: [tại đây](https://www.nasdaq.com/)**

---

# II/ Khám phá dữ liệu:

### Import

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns
import pandas as pd
import numpy as np
import tensorflow
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM


sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
%matplotlib inline

### 2.1 Đọc dữ liệu từ file csv vào dataframe

In [ ]:
df = pd.read_csv("stock_data.csv", sep = '\t')
df.head()

### 2.2 Thông tin thuộc tính:

In [ ]:
#xem thông tin data:
df.info()

num_rows, num_cols = df.shape
print("Dữ liệu có {} dòng, {} cột.".format(num_rows, num_cols))

Trước khi đi vào phân tích, ta cần hiểu về một số khái niệm trong chứng khoáng: Thị trường chứng khoáng có thể phân làm 2 loại cơ bản là **thị trường cổ phiếu (Stock market)** và **thị trường trái phiếu (bond market)**. Trong đồ án này, ta chỉ khai thác và phân tích về thị trường cổ phiếu. 

Thông tin thuộc tính:
- **Date**: Thời gian (theo ngày) của dữ liệu cổ phiếu.
- **Colse/Last**: Đây là mức giá cuối cùng mà chứng khoán được giao dịch trong một thời kỳ. Do tính lợi ích của nó, mức giá Close là một trường được thường xuyên sử dụng nhất để phân tích. Các kỹ thuật viên cho rằng mối quan hệ giữa mức giá Open (Mức giá đầu tiên) và mức giá Close (Mức giá cuối cùng) rất có ý nghĩa. Mối quan hệ đó được nhấn mạnh trong biểu đồ giá đỡ nến (candlestick chart).
- **Volume**: Đây là số lượng cổ phiếu (hoặc hợp đồng)được giao dịch trong một ngày. Mối quan hệ giữa giá và khối lượng (ví dụ: tăng giá đi cùng với tăng khối lượng) rất quan trọng.
- **Open**: Đây là mức giá của lần giao dịch đầu tiên trong một thời kỳ (ví dụ: lần giao dịch đầu tiên trong ngày). Khi phân tích các dữ liệu hàng ngày giá Open đặc biệt quan trọng vì nó là mức giá nhất trí sau khi tất cả các bên quan tâm có thể “gác nó đến hôm sau”.
- **High**: Đây là mức giá cao nhất mà chứng khoán được giao dịch trong một thời kỳ. Đó là thời điểm mà tại đó có nhiều người bán hơn người mua (tức là: luôn có người bán sẵn sàng bán ở mức giá cao hơn, nhưng mức giá High miêu tả mức giá cao nhất mà người mua sẵn sàng trả tiền).
- **Low**: Đây là mức giá thấp nhất mà chứng khoán được giao dịch trong một giai đoạn. Đó là thời điểm mà tại đó có nhiều người mua hơn người bán (tức là: luôn có người mua sẵn sàng mua ở mức giá thấp hơn, nhưng mức giá Low miêu tả mức giá thấp nhất mà người bán sẵn sàng chấp nhận bán).
- **Company**: Tên công ty ứng với mã cổ phiếu.

Chi tiết tham khảo: [tại đây](https://hotroontap.com/phan-tich-ky-thuat-bang-bieu-do-chung-khoan/)

---

## 2.3 Tiền xử lý:

- Kiểm tra dữ thông tin dữ liệu: kiểu dữ liệu của các thuộc tính, các cột có bị thiếu giá trị hay không?...

In [ ]:
#xem thông tin data:
df.info()

if df.isnull().values.any() == False:
    print("=> Không có ô giá trị nào bị thiếu.")
else:
    print("Dữ liệu bị thiếu.")

- Định dạng lại kiểu dữ liệu cho phù hợp.

In [ ]:
df_pre = df.copy()
#Định dạng lại cột Date sang kiểu dữ liệu datetime
df_pre["Date"] = pd.to_datetime(df_pre["Date"])

#Xóa khoảng trắng trong dữ liệu:
df_pre.columns = df_pre.columns.str.replace(' ', '') 
df_pre.replace(' ', '', regex=True, inplace = True)

#Đổi tên cột để tiện thao tác
df_pre.rename(columns = {'Close/Last':'Close'}, inplace = True)

#Chuyển cột có ký hiệu tiền tệ ($) sang dạng datetime:
df_pre['Close'] = df_pre['Close'].str.replace('$', '').astype(float)
df_pre['Open'] = df_pre['Open'].str.replace('$', '').astype(float)
df_pre['High'] = df_pre['High'].str.replace('$', '').astype(float)
df_pre['Low'] = df_pre['Low'].str.replace('$', '').astype(float)

#Sắp xếp cổ phiếu tăng dần theo ngày:
df_pre.sort_values(['Company', 'Date'], ascending=[True, True], inplace = True)

#Lấy tên tất cả công ty trong dữ liệu:
company_name = list(df_pre["Company"].unique())

- Kiểm tra tính liên tục của cột dữ liệu ngày, xem có bị thiếu hụt giá trị hay không:

In [ ]:
mising_date = pd.date_range(start = df_pre.Date.min(), end = df_pre.Date.max(), freq = 'D').difference(df_pre.Date)
print("Có {} ngày bị thiếu trong bộ dữ liệu.".format(len(mising_date)))

-> Do đó ta cần bổ sung phần dữ liệu bị thiếu bằng phương pháp tính trung bình giá trị không thiếu gần nhất ở trước và sau:

In [ ]:
data = pd.DataFrame(columns=df_pre.columns)
companies = df_pre.loc[:, 'Company'].unique()
for company in companies:
    one_company_df = df_pre.loc[df_pre.loc[:, 'Company'] == company]
    one_company_df.loc[:, 'Date'] = pd.to_datetime(one_company_df.loc[:, 'Date'])
    one_company_df = one_company_df.sort_values(by=['Date'], ascending=True)
    one_company_df.set_index('Date', inplace=True)

    r = pd.date_range(start=one_company_df.index.min(), end=one_company_df.index.max())
    one_company_df.index = pd.DatetimeIndex(one_company_df.index)
    one_company_df = one_company_df.reindex(r, fill_value=0)
    one_company_df = one_company_df.reset_index().rename(columns={'index': 'Date'})
    one_company_df = one_company_df.sort_values(by=['Date'], ascending=False).reset_index()
    one_company_df.loc[:, 'Company'] = company
    column_lists = ['Close', 'Volume', 'Open', 'High', 'Low']
    for column in column_lists:
        temp = one_company_df.loc[:, column]
        prev = 0
        nxt = 0
        i = 0
        while i < len(temp):
            if temp.iloc[i] != 0:
                prev = temp[i]
                i += 1
            else:
                for j in range(i, len(temp)):
                    if temp.iloc[j] != 0:
                        nxt = temp.iloc[j]
                        temp.iloc[i:j] = (prev + nxt) / 2
                        i = j
                        break
        one_company_df.loc[:, column] = temp
    data = data.append(one_company_df)

data = data.reset_index().drop(columns=['level_0', 'index'])
data = data.set_index('Date')

- Dữ liệu sau khi tiền xử lý:

In [ ]:
data.info()
data.head()

In [ ]:
# Summary Stats
data.describe()

---
# III/ Đặt câu hỏi:
- 1. Lợi tức hàng ngày của cổ phiếu trung bình là bao nhiêu?

- 2. Chúng ta phải chịu rủi ro bao nhiêu khi đầu tư vào một cổ phiếu cụ thể?

- 3. Tác động của đại dịch Covid-19 đến tình hình kinh tế của các công ty?

- 4. 

- 5. 


---

# IV/ Tìm câu trả lời:

1. **<font color=red>Tác động của đại dịch Covid-19 đến tình hình kinh tế của các công ty? </font>**

- Đầu tiên cần thực hiện bước tiền xử lý nhằm lấy ra dữ liệu của 1 khoảng thời gian trong tập:

In [ ]:
def get_records_in_time_period(df, start_date, end_date, company_name=None):
    '''
    Hàm get_records_in_time_period sẽ lấy các records của công ty trong một khoảng thời gian từ một dataframe
    Tham số df: dataframe nguồn chứa các records.
    Tham số start_date: mốc thời gian bắt đầu.
    Tham số end_date: mốc thời gian kết thúc.
    Tham số company_name: tên công ty cần lấy record, mặc định None sẽ lấy toàn bộ.
    Output: một dataframe chứa các records với tất cả các cột dữ liệu.
    '''
    # nếu không truyền vào tên công ty, mặc định sẽ lấy toàn bộ các công ty trong khoảng thời gian
    if company_name is None:
        return df.loc[(df.index >= start_date) & (df.index <= end_date)]
    # ngược lại, nếu có tham số tên công ty được truyền vào, chỉ lấy các dòng của công ty đó
    else:
        company_df = df[df['Company'] == company_name]
        time_period_filter = (company_df.index >= start_date) & (company_df.index <= end_date)
        return company_df.loc[time_period_filter]

- Hiển thị biểu đồ đường giá Close của cổ phiếu trong giai đoạn đầu năm 2020 đến nay:

In [ ]:
plt.figure(figsize=(24,12))
plt.title('Prices of all the stocks')
plt.ylabel('stock price')
plt.xlabel('date')

locs, labels = plt.xticks()
print(labels)
print(locs)

for company in company_name:
    data_company = get_records_in_time_period(data, '2020-01-01', '2020-11-30', company_name=company)
    plt.plot(data_company.index,data_company.Close, label=company)
    plt.legend()

**Nhận xét:** Ta thấy giá cổ phiếu các công ty đều tụt dốc vào giai đoạn đầu tháng 3, lúc dịch Covid bắt đầu bùng phát và lan nhanh ở nhiều nơi trên thế giới. Tuy nhiên tình hình ổn định hơn kể từ tháng 4, các công ty thương mại điện tử như Amazon tăng trưởng vượt bậc do nhu cầu mua sắm online trong thời kỳ cách ly. 

2. **<font color=red>Tác động của đại dịch Covid-19 đến tình hình kinh tế của các công ty? </font>**

In [ ]:
# https://www.kaggle.com/kratisaxena/stock-market-technical-indicators-visualization/notebook
data_test = get_records_in_time_period(data, '2020-01-01', '2020-11-30').copy()
data_test['HL'] = data_test['High'] - data_test['Low']
data_test['absHC'] = abs(data_test['High'] - data_test['Close'].shift(1))
data_test['absLC'] = abs(data_test['Low'] - data_test['Close'].shift(1))
data_test['TR'] = data_test[['HL','absHC','absLC']].max(axis=1)
data_test['ATR'] = data_test['TR'].rolling(window=14).mean()
data_test['NATR'] = (data_test['ATR'] / data_test['Close']) *100
data_test.fillna(0)

In [ ]:
plt.style.use('ggplot')
fig = plt.figure(figsize=(20,25))
for i in range(6):
    data_temp = data_test[data_test['Company'] == company_name[i]]
    ax = plt.subplot(3,2,i+1)
    ax.plot(data_temp.index, data_temp['NATR'], 'red', label="NATR", alpha = 0.5)
    ax.plot(data_temp.index, data_temp['Close'], 'cyan', label="Close", alpha = 0.5)
    ax.set_title("NATR of " + company_name[i])
    ax.legend()
    ax.set_xlabel("Date")
    ax.set_ylabel("Close Price")
    plt.xticks(rotation=30)
fig.tight_layout()

4. **<font color=red>Kiểm chứng model LSTM áp dụng cho giá cổ phiếu của Alphabet bằng dữ liệu ta có. Từ đó dự đoán giá cổ phiếu cho ngày tiếp theo sau ngày cuối trong dữ liệu? </font>**

Để tạo mô hình huấn luyện, bước đầu tiên ta cần phải chuẩn bị dữ liệu cho mô hình. Ở đây, ta sẽ làm việc với cột giá trị stock lúc đóng phiên giao dịch của Alphabet. Sau đó chuẩn hóa dữ liệu bằng `MinMaxScaler` của sklearn. 

In [ ]:
src_stock = data[data['Company'] == 'Alphabet']['Close'].iloc[::-1]
alpha_stock = src_stock.values
alpha_stock = alpha_stock.reshape(len(alpha_stock), 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_stock = scaler.fit_transform(alpha_stock)
scaled_stock

Với dữ liệu này, ta lấy 20% dòng cuối để làm giá trị test, còn lại ta dùng cho việc huấn luyện mô hình.

In [ ]:
train_values = scaled_stock[:int(len(alpha_stock) * 0.8)]
train_values

Sau đó, tạo ra tập X_train và Y_train.
- Với Y_train, ta lấy giá trị stock của một ngày.
- Với X_train, ta lấy giá trị của 100 dòng giá trị trước ngày đó.

In [ ]:
X_train, Y_train = [], []

for i in range(100, len(train_values)):
    X_train.append(train_values[i - 100:i])
    Y_train.append(train_values[i])

In [ ]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

Bắt đầu xây dựng mô hình huấn luyện. Các dòng có ý nghĩa như sau:
- Dòng 1: khởi tạo model.
- Dòng 2: thêm layer LSTM vào model. Layer này có các tham số như `units` = 128 là kích thước chiều của output, `return_sequences=True` để trả về output, `input_shape` để xác định chiều của input.
- Dòng 3: thêm layer LSTM thứ 2. Layer này có 64 units và không trả về output.
- Dòng 4: thêm layer `Dense`. Layer này có tham số là 25, nghĩa là số chiều của output là 25.
- Dòng 5: thêm layer `Dense` thứ 2. Layer này có tham số là 1, nghĩa là output chỉ còn 1 chiều.
- Dòng 6: compile model với tham số `optimizer` để tạo metrics tính toán và `loss` để model xác định cách giảm thiểu lỗi.
- Dòng 7: fit model với các tập train. Tham số `batch_size` để xác định số lượng sample mỗi lần update gradient, tham số `epochs` xác định kích thước epoch để train model.

Việc lặp các layer để tăng độ phức tạp và không gian của model, từ đó có thể có được kết quả train tốt hơn.

In [ ]:
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (X_train.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, Y_train, batch_size=1, epochs=1)

Sau khi tạo và huấn luyện mô hình cho tập huấn luyện, ta tạo dữ liệu cho tập test. Ta lấy thêm 100 dòng từ dữ liệu train để có thể dự đoán giá trị đầu tiên trong tập test.

In [ ]:
test_values = scaled_stock[int(len(alpha_stock) * 0.8) - 100:]

In [ ]:
X_test = []
Y_test = alpha_stock[int(len(alpha_stock) * 0.8):, :]

for i in range(100, len(test_values)):
    X_test.append(test_values[i - 100:i])

X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1 ))


Áp dụng mô hình cho tập dữ liệu test.

In [ ]:
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)

Tính độ lỗi RMSE cho tập dự đoán và tập test.

In [ ]:
rmse = np.sqrt(np.mean(((predictions - Y_test) ** 2)))
rmse

Từ các tập dữ liệu huấn luyện, tập dự đoán và tập kết quả, ta vẽ đồ thị để dễ dàng quan sát.

In [ ]:
src_stock

In [ ]:
train_set = src_stock.iloc[:int(0.8 * len(src_stock))]
test_set = src_stock.iloc[int(0.8 * len(src_stock)):]
prediction_set = pd.Series(predictions.reshape(len(predictions)))
prediction_set.index = test_set.index

In [ ]:
prediction_set

In [ ]:
test_set

In [ ]:
fig = plt.figure(figsize=(15, 5))
plt.xlabel('Date', color="red")
plt.ylabel('Price', color="red")
plt.plot(train_set)
plt.plot(test_set)
plt.plot(prediction_set)
plt.tick_params(axis='x', colors="C0")
plt.tick_params(axis='y', colors="C0")
plt.legend(['Train set', 'Test set', 'Prediction set'], loc='upper left')
plt.show()

#### Nhận xét: 
- Từ độ lỗi và biểu đồ trực quan, ta có thể thấy mô hình LSTM này khá phù hợp với việc dự đoán giá cổ phiếu, chủ yếu là vì mô hình này phù hợp với loại dữ liệu liên tục và có sự kết giữa các mốc giá trị.
- Đối với những ngày có sự thay đổi không lớn (không đột ngột), mô hình dự đoán khá chính xác, tuy nhiên, với những ngày có giá biến động thất thường, mô hình cho ra kết quả dự đoán tương đối lệch. Điều này là hiển nhiên không thể tránh khỏi, vì mô hình LSTM mà ta đang sử dụng tương đối đơn giản, dữ liệu mà ta sử dụng cũng vậy, chỉ có vài cột dữ liệu. Bên cạnh đó, sự thay đổi trong giá cổ phiếu còn phụ thuộc vào rất nhiều yếu tố ngoài đời thực khác (như đại dịch, chủ tịch qua đời, sản phẩm mới ra mắt thành công hoặc thất bại, vướng phải kiện tụng,...). ***Vì thế, việc sử dụng mô hình này để dự đoán chỉ mang tính chất tham khảo.***

Xây dựng hàm `lstm_model` để có thể áp dụng cho nhiều công ty.
- Hàm này yêu cầu tham số là dataframe dữ liệu, tên công ty và một số nguyên là giá trị độ dài một dòng input mà ta mong muốn.
- Cách hoạt động của hàm:
    - Chia tập dữ liệu thành 2 phần: 80% đầu dành cho việc train và 20% còn lại dành cho việc test.
    - Với mỗi giá trị cổ phiếu, ta lấy giá trị cổ phiếu của N ngày trước đó để làm input cho việc huấn luyện, và giá trị ngày đó làm output.
    - Tiếp tục lặp với toàn bộ tập train để có được model.
- Sau khi hoàn thành việc huấn luyện mô hình, hàm sẽ trả về một vài giá trị như:
    - `model`: mô hình LSTM đã huấn luyện.
    - `scaler`: mô hình chuẩn hóa dữ liệu.
    - `rmse`: độ lỗi trên tập test.

In [ ]:
def lstm_model(df, company_name, input_len=100):
    '''
    Hàm lstm_model sẽ tạo một LSTM model đã qua việc train với 80% lượng dữ liệu ta có.
    Tham số df: dataframe chứa dữ liệu để tạo model.
    Tham số company_name: tên công ty để lấy dữ liệu.
    Tham số input_len: độ dài của một dòng input.
    Output: một model LSTM đã qua việc huấn luyện, mô hình chuẩn hóa dữ liệu, độ lỗi trên tập test.
    '''
    
    stock = df[df['Company'] == company_name]['Close'].values
    stock = stock.reshape(len(stock), 1)
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_stock = scaler.fit_transform(stock)

    train_values = scaled_stock[:int(len(stock) * 0.8)]
    X_train, Y_train = [], []
    
    for i in range(input_len, len(train_values)):
        X_train.append(train_values[i - input_len:i])
        Y_train.append(train_values[i])

    X_train = np.array(X_train)
    Y_train = np.array(Y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    
    # tạo mô hình LSTM để huấn luyện
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, Y_train, batch_size=1, epochs=1)
    
    test_values = scaled_stock[int(len(stock) * 0.8) - input_len:]
    X_test = []
    Y_test = stock[int(len(stock) * 0.8):, :]

    for i in range(input_len, len(test_values)):
        X_test.append(test_values[i - input_len:i])

    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1 ))

    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)
    
    rmse = np.sqrt(np.mean(((predictions - Y_test) ** 2)))

    return model, scaler, rmse
